# tensorflow 实现简单线性回归

In [ ]:
# tensorflow environment
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt

# refer to d2l
from d2l import tensorflow as d2l

# using type hint from third party tools
# deprecated since python3.9
# from typing import list

# ensure GPU available
print(tf.config.list_physical_devices("GPU"))

## 生成数据

In [ ]:
def synthetic_data(w:list[float],b:float,num:int):
    """
    creat data
    """
    x = tf.zeros((num,w.shape[0]))
    x += tf.random.normal(shape=x.shape) # noise
 
    y= tf.matmul(x,tf.reshape(w,(-1,1))) + b
    y += tf.random.normal(shape=y.shape,stddev=0.01)
    y = tf.reshape(y,(-1,1))
    return x,y


In [ ]:
# weights
w = tf.constant([1,2.1,3],dtype=float)
# bias
b = -1
n = 1000

features,lables=synthetic_data(w,b,n)
print("features:",features[0])
print("labels:",lables[0])

## 读取数据

In [ ]:
def data_iter(batch_size,features,lables):
    """通过yield生成迭代器"""
    nums=len(features)
    # 随机读取样本
    indices = list(range(nums))
    random.shuffle(indices)
    for i in range(0,nums,batch_size):
        j = tf.constant(indices[i:min(i+batch_size,nums)])
        yield tf.gather(features,j),tf.gather(lables,j)
    pass

In [ ]:
batch_size = 10

for x,y in data_iter(batch_size,features,lables):
    print(x,"\n")
    print(y)

## 建立线性回归模型

In [ ]:
# 初始化模型参数
tw = tf.Variable(tf.random.normal(shape=(3,1),mean=0,stddev=0.01),trainable=True)
tb = tf.Variable(tf.zeros(1))

In [ ]:
# 定义模型
def liner_regression(x,w,b):
    """线性回归模型"""
    return tf.matmul(x,w)+b

## 选择合适的损失函数和优化器

In [ ]:
# 损失函数
def squared_loss(y_hat,y):
    """均方损失"""
    return (y_hat - tf.reshape(y,y_hat.shape)) ** 2 / 2

In [ ]:
# 优化算法
def sgd(params, grads,lr,batch_size):
    """小批量随机梯度下降"""
    for param, grad in zip(params, grads):
        param.assign_sub(lr*grad/batch_size)

In [ ]:
# 训练及参数


lr = 0.03
num_epochs = 10
net = liner_regression
loss = squared_loss

for epoch in range(num_epochs):
    for _x,_y in data_iter(batch_size=batch_size,features=x,lables=y):
        with tf.GradientTape() as g:
            # print(_x,w)
            l = loss(net(_x,tw,tb),_y)
        dw,db=g.gradient(l,[tw,tb])
        sgd([tw,tb],[dw,db],lr,batch_size)
    train_l = loss(net(features,tw,tb),lables)
    print(f'epoch {epoch + 1}, loss {float(tf.reduce_mean(train_l)):f}')




In [ ]:
print(f'w的估计误差: {w - tf.reshape(tw, w.shape)}')
print(f'b的估计误差: {b - tb}')